In [10]:
import gc
import json
import numpy as np
import pandas as pd

from boruta import BorutaPy
from glob import glob
#from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
#from sklearn.feature_selection import RFECV
from tqdm import tqdm

from utils import line_notify, FEATS_EXCLUDED, removeCorrelatedVariables, removeMissingVariables

%matplotlib inline

In [34]:
configs = json.load(open('../configs/200_all.json'))

# load feathers
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

# set card_id as index
df.set_index('card_id', inplace=True)

  0%|          | 0/1101 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
100%|██████████| 1101/1101 [00:01<00:00, 718.76it/s]


In [35]:
df = df[configs['features']]
df

,AOV,AOV_ratio,CLV,CLV_month_diff,CLV_month_diff_ratio,CLV_ratio,Mothers_Day_2018,Outlier_Likelyhood,amount_month_ratio_min,amount_month_ratio_ratio,...,observation_date_ratio,outliers,pred_CLV_ratio,price_max,price_mean,price_ratio,purchase_amount_max,target,weekofyear,year
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,182.318395,1.661881,4.690294e+06,1.705895e+08,-0.008053,0.013005,49.492977,0.000273,-350.000000,NaN,...,0.831091,0.0,0.043402,1329.0000,11856.00000,NaN,1329.000,-0.820283,0.012435,0.007631
C_ID_3d0044924f,101.445060,0.159410,1.071890e+07,1.005798e+07,0.000050,0.000047,56.833333,0.001651,7.990000,0.000000,...,0.785113,0.0,0.000052,96.2500,39.53125,0.332394,1059.000,0.392913,0.017699,0.007631
C_ID_d639edf6cd,76.651163,0.679063,8.444000e+04,NaN,NaN,0.000367,15.720930,0.003357,NaN,NaN,...,0.831091,0.0,0.000000,NaN,NaN,NaN,431.000,0.688056,0.009683,0.015914
C_ID_186d6a6901,118.311688,0.856153,3.805900e+05,NaN,NaN,0.007076,55.623377,0.001447,NaN,NaN,...,0.831091,0.0,0.014900,287.2500,55.12500,0.906318,1149.000,0.142495,0.006126,0.007631
C_ID_cdbd2c0db2,249.208020,1.067407,2.299008e+06,-9.753155e+07,0.031752,0.078204,61.856099,0.000277,-156.090000,NaN,...,0.831091,0.0,0.131382,130.3750,104.00000,1.406941,1826.000,-0.159749,0.007097,0.007631
C_ID_0894217f2f,483.522727,1.205120,2.430160e+05,3.456953e+04,0.017783,0.017706,0.000000,0.026964,15.214286,0.146393,...,0.895487,0.0,0.024892,67.7500,84.06250,1.055530,1016.500,0.871585,0.006126,0.015914
C_ID_7e63323c00,40.024038,0.781892,1.518839e+06,NaN,NaN,0.000289,60.980769,0.001925,NaN,NaN,...,0.831091,0.0,0.000761,NaN,NaN,NaN,169.500,0.230129,0.008807,0.015914
C_ID_dfa21fc124,118.265152,0.364359,4.223825e+04,2.111912e+04,0.006775,0.006775,87.333333,0.003648,15.500000,0.049691,...,0.840854,0.0,0.002206,NaN,NaN,NaN,493.000,2.135850,0.006126,0.007631
C_ID_fe0fdac8ea,187.483333,0.693107,2.522200e+04,NaN,NaN,0.012322,54.500000,0.005695,NaN,NaN,...,0.831091,0.0,0.001268,146.8750,90.75000,0.769712,440.500,-0.065406,0.009683,0.007631


In [36]:
imp = pd.read_csv('../output/feature_importance_lgbm.csv')
imp

,feature,importance,fold
0,AOV,54.825733,15
1,AOV_ratio,67.639101,15
2,CLV,55.132264,15
3,CLV_month_diff,54.085351,15
4,CLV_month_diff_ratio,53.826306,15
5,CLV_ratio,61.812106,15
6,Mothers_Day_2018,54.003377,15
7,amount_month_ratio_min,57.368250,15
8,amount_month_ratio_ratio,59.914604,15
9,card_id_cnt_ratio,56.081048,15


In [37]:
#col_zero_imp = imp['feature'][imp.importance==0].tolist()
df.drop(col_zero_imp, axis=1, inplace=True)

KeyError: "['hist_amount_month_ratio_unapproved_min' 'hist_duration_unapproved_min'\n 'hist_installments_min' 'hist_installments_unapproved_min'\n 'hist_mer_active_months_lag12_skew_mean'\n 'hist_mer_active_months_lag12_skew_sum'\n 'hist_mer_active_months_lag3_max_sum'\n 'hist_mer_active_months_lag3_mean_mean'\n 'hist_mer_active_months_lag3_mean_sum'\n 'hist_mer_active_months_lag3_min_sum'\n 'hist_mer_active_months_lag3_skew_mean'\n 'hist_mer_active_months_lag3_skew_sum'\n 'hist_mer_active_months_lag3_var_sum'\n 'hist_mer_active_months_lag6_min_sum'\n 'hist_mer_active_months_lag6_skew_mean'\n 'hist_mer_active_months_lag6_skew_sum'\n 'hist_mer_avg_active_months_skew_mean'\n 'hist_mer_avg_active_months_skew_sum' 'hist_mer_avg_numerical_skew_mean'\n 'hist_mer_avg_numerical_skew_sum'\n 'hist_mer_avg_purchases_lag12_skew_mean'\n 'hist_mer_avg_purchases_lag12_skew_sum'\n 'hist_mer_avg_purchases_lag3_skew_sum'\n 'hist_mer_avg_purchases_lag6_skew_mean'\n 'hist_mer_avg_purchases_lag6_skew_sum' 

In [27]:
# remove missing variables
col_missing = removeMissingVariables(df,0.75)
df.drop(col_missing, axis=1, inplace=True)

Removing Missing Variables...


In [29]:
# remove correlated variables
col_drop = removeCorrelatedVariables(df,0.9)
df.drop(col_drop, axis=1, inplace=True)

Removing Correlated Variables...


In [38]:
col_drop = pd.Series(col_drop)

In [30]:
df

,AOV,AOV_ratio,CLV,CLV_month_diff,CLV_month_diff_ratio,CLV_ratio,Mothers_Day_2018,Outlier_Likelyhood,amount_month_ratio_min,amount_month_ratio_ratio,...,observation_date_ratio,outliers,pred_CLV_ratio,price_max,price_mean,price_ratio,purchase_amount_max,target,weekofyear,year
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,182.318395,1.661881,4.690294e+06,1.705895e+08,-0.008053,0.013005,49.492977,0.000273,-350.000000,NaN,...,0.831091,0.0,0.043402,1329.0000,11856.00000,NaN,1329.000,-0.820283,0.012435,0.007631
C_ID_3d0044924f,101.445060,0.159410,1.071890e+07,1.005798e+07,0.000050,0.000047,56.833333,0.001651,7.990000,0.000000,...,0.785113,0.0,0.000052,96.2500,39.53125,0.332394,1059.000,0.392913,0.017699,0.007631
C_ID_d639edf6cd,76.651163,0.679063,8.444000e+04,NaN,NaN,0.000367,15.720930,0.003357,NaN,NaN,...,0.831091,0.0,0.000000,NaN,NaN,NaN,431.000,0.688056,0.009683,0.015914
C_ID_186d6a6901,118.311688,0.856153,3.805900e+05,NaN,NaN,0.007076,55.623377,0.001447,NaN,NaN,...,0.831091,0.0,0.014900,287.2500,55.12500,0.906318,1149.000,0.142495,0.006126,0.007631
C_ID_cdbd2c0db2,249.208020,1.067407,2.299008e+06,-9.753155e+07,0.031752,0.078204,61.856099,0.000277,-156.090000,NaN,...,0.831091,0.0,0.131382,130.3750,104.00000,1.406941,1826.000,-0.159749,0.007097,0.007631
C_ID_0894217f2f,483.522727,1.205120,2.430160e+05,3.456953e+04,0.017783,0.017706,0.000000,0.026964,15.214286,0.146393,...,0.895487,0.0,0.024892,67.7500,84.06250,1.055530,1016.500,0.871585,0.006126,0.015914
C_ID_7e63323c00,40.024038,0.781892,1.518839e+06,NaN,NaN,0.000289,60.980769,0.001925,NaN,NaN,...,0.831091,0.0,0.000761,NaN,NaN,NaN,169.500,0.230129,0.008807,0.015914
C_ID_dfa21fc124,118.265152,0.364359,4.223825e+04,2.111912e+04,0.006775,0.006775,87.333333,0.003648,15.500000,0.049691,...,0.840854,0.0,0.002206,NaN,NaN,NaN,493.000,2.135850,0.006126,0.007631
C_ID_fe0fdac8ea,187.483333,0.693107,2.522200e+04,NaN,NaN,0.012322,54.500000,0.005695,NaN,NaN,...,0.831091,0.0,0.001268,146.8750,90.75000,0.769712,440.500,-0.065406,0.009683,0.007631


In [33]:
# save feature name list
features_json = {}
features_json['features'] = df.columns.tolist()
with open('../configs/200_all.json', 'w') as f:
    json.dump(features_json, f, indent=4)

In [40]:
col_drop.to_csv('../output/col_correlated.csv')

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
